# EE4375-2022: Third Lab Session: Using DiffEqOperators in Two Dimensions

This example is taken from [https://rveltz.github.io/BifurcationKit.jl/stable/tutorials2/]. Seems that interior nodes only are taken into account. 

In [32]:
using DiffEqOperators
using LinearAlgebra, Plots, SparseArrays

# helper function to plot solution
#heatmapsol(x) = heatmap(reshape(x,Nx,Ny)',color=:viridis)

Nx = 2
Ny = 2 
hx = 1/Nx
hy = 1/Ny 

# we use DiffEqOperators to compute the Laplacian operator
function Laplacian2D(Nx, Ny, hx, hy)
	D2x = CenteredDifference(2, 2, hx, Nx)
	D2y = CenteredDifference(2, 2, hy, Ny)
	Qx = Neumann0BC(hx)
	Qy = Neumann0BC(hy)
	Qx = Dirichlet0BC(Float64)
    Qy = Dirichlet0BC(Float64)
	A = kron(sparse(I, Ny, Ny), sparse(D2x * Qx)[1]) + kron(sparse(D2y * Qy)[1], sparse(I, Nx, Nx))
	return A
end

A = Laplacian2D(Nx, Ny, hx, hy)

print(A)


 -16.0    4.0    4.0     ⋅ 
   4.0  -16.0     ⋅     4.0
   4.0     ⋅   -16.0    4.0
    ⋅     4.0    4.0  -16.0

In [28]:
using Test
using DiffEqOperators, OrdinaryDiffEq, LinearAlgebra

# Space domain and grids
N = 3
Δx = 1. #/(N-1)
c = 1
x = 0:Δx:1;
t = 0. 

# solution of the single forward moving wave
ϕ(x,t) = 1. #(1/2)*sech.((x .- t)/2).^2 

# Discretizing the PDE at t = 0
u0 = ones(size(x));
du = zeros(size(x)); 

# Declaring the Upwind operator with winding = -1 since the wave travels from left to right 
A = UpwindDifference{Float64}(1,3,Δx,length(x),-1);

# Qx = Neumann0BC(Δx)
# Qx = NeumannBC((2.,2.),Δx)
Qx = NeumannBC((2.,5.),Δx) # Left and right node 

# for Robin boundary conditions
# use two (left and right) tuples of three (alpha, beta and gamma). 

bc = GeneralBC([0,1,-6*ϕ(-10,t),0,-1],[0,1,-6*ϕ(10,t),0,-1],Δx,3)

y0 = Qx*u0 

print("Prior handling bc ", u0,"\n\n")
print("After handling bc ", y0,"\n\n")

Prior handling bc [1.0, 1.0]

After handling bc [-1.0, 1.0, 1.0, 6.0]



In [8]:
typeof(bc)

GeneralBC{Float64, Vector{Float64}, Vector{Float64}}

In [ ]:
using ModelingToolkit, MethodOfLines, DomainSets, NonlinearSolve

# Define the system of equations
@parameters x y
@variables u(..)

Dxx = Differential(x)^2
Dyy = Differential(y)^2

xmin = 0.
xmax = 1.
ymin = 0.
ymax = 1.

f(x,y) = x^2 + y^2 #If you have array data, define a piecewise function or use an interpolation

eq = Dxx(u(x,y)) + Dyy(u(x,y)) + u(x,y) ~ f(x,y)

domain = [x ∈ Interval(xmin,xmax), 
          y ∈ Interval(ymin,ymax)]

# Arbitrary bcs can be defined in a natural way
# Dirichlet 0 would be u(xmin,y) ~ 0.
# Neumann would be Differential(x)(u(xmin,y)) ~ α(x, y) where α is a function, can be replaced with a constant value.
# Periodic would be u(xmin,y) ~ u(xmax,y)
bcs = [u(xmin, y) ~ u(xmax, y),
       u(x, ymin) ~ u(x, ymax)]

@named pdesys = PDESystem(eq, bcs, domain, [x, y], [u(x,y)])

# Create discretization

dx = 0.01
dy = 0.01

order = 2 # Order of the finite element approximation, must be even

discretization = MOLFiniteDifference([x => dx, y => dy], approx_order = order, grid_align = center_align) # edge_align gives better results with Neumann BCs, though uses extra grid points

prob = discretize(pdesys, discretization)

sol = NonlinearSolve.solve(prob, NewtonRaphson())

# Center aligned grid
disc_x = xmin:dx:xmax
disc_y = ymin:dy:ymax

# Edge aligned grid
# disc_x_e = xmin-dx/2:dx:xmax+dx/2
# disc_y_e = ymin-dy/2:dy:ymax+dy/2

sol_u = reshape(sol.u, (length(disc_x), length(disc_y)))

# Plot the solution
using Plots
heatmap(disc_x, disc_y, sol_u)